## Importing SQL Table via SparkSQL

## Preliminary EDA with SparkSQL

### Checking total number of records and column names



In [0]:
# Number of records
print("Total records:", df.count())

# Column list
print("Columns:", df.columns)

### Analyzing class distribution for delivery status (on time vs delayed)


In [0]:
# Count of each class in Reached.on.Time_Y.N
df.groupBy("`Reached.on.Time_Y.N`").count().show()


### Viewing summary statistics for numeric features


In [0]:
# Describe numeric fields like Weight_in_gms and Customer_rating
df.select("Weight_in_gms", "Customer_rating").describe().show()


### Exploring delivery delay distribution across warehouse blocks and modes of shipment


In [0]:
# Delays grouped by warehouse block
df.groupBy("Warehouse_block", "`Reached.on.Time_Y.N`").count().orderBy("Warehouse_block").show()

# Delays grouped by mode of shipment
df.groupBy("Mode_of_Shipment", "`Reached.on.Time_Y.N`").count().orderBy("Mode_of_Shipment").show()


### Checking average shipment weight based on delivery delay status


In [0]:
from pyspark.sql.functions import avg

df.groupBy("`Reached.on.Time_Y.N`").agg(avg("Weight_in_gms").alias("Avg_Weight")).show()


### What Did We Learn from the Data?

- A majority of deliveries (~60%) were delayed (`6563` out of `10999`).
- Weight appears to correlate with delays: on-time deliveries have a higher average weight (~4168g vs ~3272g).
- Warehouse F has the highest number of both on-time and delayed deliveries, suggesting it handles more volume overall.
- Shipments by ship show a higher delay count compared to flights and roads — could indicate slower or more unreliable logistics mode.
- Customer ratings are roughly centered around 3 (mean ~2.99), so they may not be a strong predictive feature, but we’ll include them for now.

## Building the ML Model

### Converting data to Pandas Dataframe for training (Databricks Community Edition)

In [0]:
selected_cols = [
    "Warehouse_block",
    "Mode_of_Shipment",
    "Product_importance",
    "Customer_rating",
    "Customer_care_calls",
    "Cost_of_the_Product",
    "Prior_purchases",
    "Discount_offered",
    "Weight_in_gms",
    "`Reached.on.Time_Y.N`"
]

# Subset Spark DataFrame and convert to Pandas
pdf = df.select(*selected_cols).toPandas()

# Preview Pandas DataFrame
pdf.head()

### Encoding Categorical Variables

In [0]:
from sklearn.preprocessing import LabelEncoder

df_model = pdf.copy()

categorical_cols = ["Warehouse_block", "Mode_of_Shipment", "Product_importance"]

le = LabelEncoder()
for col in categorical_cols:
    df_model[col] = le.fit_transform(df_model[col])

### Seperating Labels and Features

In [0]:

X = df_model.drop("Reached.on.Time_Y.N", axis=1)
y = df_model["Reached.on.Time_Y.N"]


### Train Test Split

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Defining Sweep Parameters for Model

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}


### Initializing Model and GridSearchCV

In [0]:

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=3,             # 3-fold cross-validation
                           scoring='f1_weighted',
                           n_jobs=-1,
                           verbose=1)


In [0]:

grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)


### Evaluating Model

In [0]:

best_rf = grid_search.best_estimator_
y_pred_tuned = best_rf.predict(X_test)

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix

print("Tuned Random Forest Classification Report:\n")
print(classification_report(y_test, y_pred_tuned))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_tuned))


### Visualizing Feature importance

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Extract and sort feature importances from the best model
tuned_feature_importances = pd.Series(best_rf.feature_importances_, index=X.columns)
tuned_feature_importances = tuned_feature_importances.sort_values(ascending=True)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=tuned_feature_importances, y=tuned_feature_importances.index, palette="magma")

plt.title("Feature Importances from Tuned Random Forest Model")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.tight_layout()
plt.show()


### Feature Importance Insights (Tuned Random Forest)

- Discount Offered and Weight in gms are the most influential features, together accounting for over 70% of the model’s predictive power. This indicates that heavier shipments and larger discounts are strongly associated with delivery delays.
- Cost of the Product and Prior Purchases also contribute meaningfully, suggesting pricing and customer loyalty may influence delivery performance.
- Features such as Mode of Shipment, Product Importance, and Customer Rating had minimal impact on predictions, implying they may be less predictive in this dataset.
